# Ejercicio Strem Cipher
Marco A. Jurado 20308

In [64]:
import random
import os

In [65]:
inputCifrado = 'Este es el mensaje que quiero cifrar'
seed = 2 # Este es el valor con el cual se va a replicar el output del keystream

## ystream

In [66]:
def get_ascii_chars():
    retorno = ''
    for i in range(32, 127):
        retorno += chr(i)
    return retorno

In [67]:
def generate_keystream(seed, length):
    asciss = get_ascii_chars() + 'ñÑ'

    random.seed(os.urandom(32) + str(seed).encode())

    keystream = ''

    for _ in range(length):
        # Seleccionar un carácter aleatorio del alfabeto ascii que tiene todos los caracteres + ñ
        keystream += random.choice(asciss)

    return keystream

G = generate_keystream(seed, len(inputCifrado))
print('Keystream generada con los valores de llave y el largo de mensaje:\n    ->', G)

Keystream generada con los valores de llave y el largo de mensaje:
    -> w0&hn(3r~O>IUme=UJlH)DP1Am;a0TñjNO8'


In [68]:
def convertir_letra_binario(x):
    a = ord(x)

    binario = ''
    while a > 0:
        residuo = a % 2
        binario = str(residuo) + binario
        a = a // 2
    temp = 8 - len(binario)

    retorno = ''
    for i in range(temp):
        retorno = retorno + '0'
    retorno = retorno + binario
    return retorno

def convertir_binario_letra(x):
    valor_decimal = 0

    for i in range(8):
        valor_decimal += int(x[i]) * (2 ** (len(x) - i - 1))


    letra = chr(valor_decimal)
    return letra

def codificar_binario(x):
    retorno = ""
    for i in x:
        retorno += convertir_letra_binario(i)
    return retorno

def decodificar_binario(x):
    retorno = ""
    for i in range(0, len(x), 8):
        retorno += convertir_binario_letra(x[i:i+8])
    return retorno

def verificar_largo_llave_xor(x,y):
    retorno = ""
    temp = 0

    for _ in range(len(x)):
        retorno += y[temp]
        temp = (temp + 1) % len(y) # modulo para asegurar que no sea más larga de la palabra

    return retorno

def coder_xor(x, y):
    y = verificar_largo_llave_xor(x, y)

    input_binario = codificar_binario(x)
    y_binario = codificar_binario(y)

    # XOR bit a bit
    result_binario = ''
    for i in range(len(input_binario)):
        temp = 0
        if input_binario[i] == '0' and y_binario[i] == '1':
            temp = 1
        elif input_binario[i] == '1' and y_binario[i] == '0':
            temp = 1

        result_binario = result_binario + str(temp)

    result_texto = decodificar_binario(result_binario)

    return result_texto

## Cifrado
Función de cifrado realizada

In [69]:
def encryption(x, key):
    input_binario = codificar_binario(x)
    key_binario = codificar_binario(key)

    key_binario = verificar_largo_llave_xor(input_binario, key_binario)

    result_binario = ''
    for i in range(len(input_binario)):
        result_binario += str(int(input_binario[i]) ^ int(key_binario[i]))

    result_texto = decodificar_binario(result_binario)
    return result_texto

outputCifrado = encryption(inputCifrado, G)
outputCifrado

'2CR\rNM@R\x1b#\x1e$0\x03\x16\\?/L9\\!p@4\x04^\x13_t\x92\x03(=YU'

## Descifrado
Dado que se hace uso de la propiedad y pipeline con la función xor se puede hacer uso de exactamente la misma funcion y se obtiene el resultado descifrado.

In [70]:
def outputDescifrado(x,y):
    return encryption(x,y)
outputDescifradotemp = outputDescifrado(outputCifrado, G)
outputDescifradotemp

'Este es el mensaje que quiero cifrar'

#### Prueba 1 - cambiando claves con el mismo texto

In [71]:
texto_prueba1 = 'Este es el texto de la prueba numero 1 que quiero cifrar'
seeds_prueba1 = [2,8,15,150,78,65132165,54646248]

for i in seeds_prueba1:
    print('con la llave ', i)
    print('    ->', generate_keystream(i, len(texto_prueba1)))

con la llave  2
    -> QO?4UM;KgnfvQ6nTc"*.Lsuj;\\;n7Jyr5+WGNd~>~,Q}0Vw]a4Ac|B 
con la llave  8
    -> pg)R IlR/ñd1ThlaivVhQ~N!@kB#^73=]l.D(;AO3XgmU-]r*G;2o<4}
con la llave  15
    -> Kl`nTO&y?!@+l3ñ\U>J*f&,zdP9DqsD,3JZ{KvnPk0}_gRpipp<!3ñ&(
con la llave  150
    ->  Ma4 V7t)8|@moL($?Lrt/'ÑuPXQi,A)/B\7\=K0,?R[)/VVÑ]dmp.#%
con la llave  78
    -> 21zR/{<d\c:3!{y;(aAwHD= F3j;#</,d]{\DpSUJA5'Ñ1k293gkTBKR
con la llave  65132165
    -> 'kp+&7AXZ-YpC"q>u" ps(9M7^/Gst;H|;_H(DJ7YZfBi&^IGa_eAh"w
con la llave  54646248
    -> ;F1FdR6zN+6~QP4{)su#XkpFYeVC.2F1iÑDbrVi}M<-;>u!v60*xr/1f


#### Prueba 2 - mismo keystream para dos mensajes

In [72]:
mensajes_prueba2 = ['Hola soy el texto 1 que vamos a probar', 'Adios mundo yo soy el ultimo texto a cifrar']
llaveprueba2 = generate_keystream(54,len(mensajes_prueba2[0]))

print('     ***** texto 1 ******')
print('llave obtenida:', llaveprueba2)
print('texto original: ', mensajes_prueba2[0])
tempprueba21 = encryption(mensajes_prueba2[0], llaveprueba2)
print('texto cifrado: ', tempprueba21)
print('texto descifrado: ', outputDescifrado(tempprueba21, llaveprueba2))

print('     ***** texto 2 ******')
print('llave obtenida:', llaveprueba2)
print('texto original: ', mensajes_prueba2[1])
tempprueba22 = encryption(mensajes_prueba2[1], llaveprueba2)
print('texto cifrado: ', tempprueba22)
print('texto descifrado: ', outputDescifrado(tempprueba22, llaveprueba2))

     ***** texto 1 ******
llave obtenida: ^sH\UoeG2%V` cK~Kc&im\kYZRÑ+mpT/ms'1z;
texto original:  Hola soy el texto 1 que vamos a probar
texto cifrado:  $=u
>@:@T3
$CI)y,3¼DP5HSI
texto descifrado:  Hola soy el texto 1 que vamos a probar
     ***** texto 2 ******
llave obtenida: ^sH\UoeG2%V` cK~Kc&im\kYZRÑ+mpT/ms'1z;
texto original:  Adios mundo yo soy el ultimo texto a cifrar
$|5.;¼DM1WPZX7:='
texto descifrado:  Adios mundo yo soy el ultimo texto a cifrar


## Preguntas
#### ¿Cómo cambia el mensaje cifrado cuando cambias la clave?
En el caso del keystream esta cambiara totalmente la llave generada al cambiar la seed o clave utilizada. Esto se debe a que al cambiar dicho valor cambiamos todas las probabilidades aleatorias del output generando así una llave completamente distinta.

#### ¿Qué sucede si utilizamos el mismo keystream para cifrar dos mensajes diferentes?
Si utilizamos el mismo keystream podemos ver que obviamente obtendresmos la misma llave y un texto cifrado distinto según sea el texto de input. Sin embargo el proceso no se ve afectado en ningun sentido.

#### ¿Cómo afecta la longitus del keystream a la seguridad del cifrado?
En el caso de una llave de mayor longitud tendremos un proceso de cifrado del mensaje de mucha más seguridad. Esto tambien quiere decir que el proceso de que la llave sea adivinada, interceptada y demás es más dificil pues se cuentan con muchos caracteres para realizar el proceso complejo, dificil y sobre todo seguro.